In [39]:
import pandas as pd
import numpy as np
import pickle
import datetime
import json
import os

### Only analyzing pilot participants

### Required Files:
- 'SurveyTrailmakingResults'
- 'SurveyStroopResults'
- 'SurveyPSATResults'
- 'SurveyTowerOfHanoiResults'
- 'SurveyReactionTimeResults'
- 'SurveySpatialSpanMemoryResults'

### Tasks that are currently relevant
- Trail making
- Stroop
- PSAT
- Tower of Hanoi
- Reaction Time
- Spatial Span Memory

### Variable Information

In [40]:
## Input Directory
directory = "../cumulative_data_811-910"

In [41]:
## Output Directory
out_dir = "../indv_table_exports/"

In [42]:
## Good Participants
good_subjects = ['01801252-3a7e-4f5f-8b6d-49e8da3902f3',
                 'd26d4b78-7fcf-488e-b687-2d1c93c47b74',
                 '531d7f6d-b880-4a0b-b467-80005a316f1c']

### Handy Functions

In [43]:
## Function to fix date from UTC to ET
def fix_date_to_ET(end_date):
    if pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').hour < 5:
        return pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date() - datetime.timedelta(days=1)
    else:
        return pd.to_datetime(end_date, format= '%Y-%m-%d', utc=True).tz_convert('US/Eastern').date()

def fix_columns_by_category(dataframe, categories):
    df = dataframe
    for item in categories:
        if item not in df.columns.to_list():
            df[item] = 'NaN'
    return df

def pivot_df(dataframe, pos: list, col, val):
    df = dataframe
    df = df.pivot_table(index=pos,
                    columns=col, 
                    values=val).reset_index()
    return df

def get_time_diff(d2, d1):
    return (d2 - d1).days

def convert_survey_file_to_dataframe(directory, filename, date_col):
    df_survey_list = []

    survey_file_name = ""
    path = directory + "/"
    
    for f_name in os.listdir(path):
        if f_name.startswith(filename):
            survey_file_name = f_name
            break

    path = path + '/' + survey_file_name

    current_df = pd.read_csv(path) 
    current_df[date_col] = current_df.apply(lambda x: fix_date_to_ET(x.EndDate), axis=1)

    ## current_df = fix_date_to_ET(current_df, date_col)
    return current_df

### Trailmaking Task
The subject is instructed to connect a set of 25 dots as quickly as possible while still maintaining accuracy. 
It measures:
- visual attention
- task switching
- fluid intelligence/cognitive abilities

We are doing a version that is like the typical Version B:
- Salthouse, 2011

_We want to get the last TapTimestamp to calculate total timing_

In [44]:
## Got all 'SurveyTrailmakingResults' till date in a dataframe
df_tr_survey = convert_survey_file_to_dataframe(directory, 
                                                'SurveyTrailmakingResults', 
                                                'ActualDueDate')

## convert string Taps to list of dicts (Wilson, D.)
df = df_tr_survey
df['TapsList'] = df['Taps'].apply(json.loads)

## assign new columns with final value from TapsList (Wilson, D.)
df = df.assign(Task_Trailmaking_Time=lambda x: x.TapsList.apply(lambda x: x[-1]['TapTimestamp']),
               Task_Trailmaking_Errors=lambda x: x.NumberOfErrors)

## keep relevant columns
df = df[['ParticipantIdentifier', 'ActualDueDate', 'Task_Trailmaking_Time', 'Task_Trailmaking_Errors']]
df_trail_making = df

### Stroop Task
- (Scarpina & Tagini, 2017)
- The Stroop Color and Word Test (SCWT) 
- Will calculate the stroop effect and relevant descriptive stats

In [45]:
## Function that returns a Series of all Stroop results aggregates (Wilson, D.)
def get_stroop_results(x):
    d = {}
    
    d['Task_Stroop_TotalCorrectProp'] = x['Correct'].sum()/len(x['Correct'])
    d['Task_Stroop_CongruentCorrectProp'] = len(x.loc[(x.Congruent==True) & (x.Correct==True)])/x.Congruent.value_counts()[1]
    d['Task_Stroop_IncongruentCorrectProp'] = len(x.loc[(x.Congruent==False) & (x.Correct==True)])/x.Congruent.value_counts()[0]
    d['Task_Stroop_TotalAvgRT'] = x['Time'].sum()/len(x['Time'])    
    d['Task_Stroop_CongruentAvgRT'] = x.loc[x.Congruent==True,'Time'].sum()/x.Congruent.value_counts()[1]    
    d['Task_Stroop_IncongruentAvgRT'] = x.loc[x.Congruent==False,'Time'].sum()/x.Congruent.value_counts()[0]
    
    return pd.Series(d, index=['Task_Stroop_TotalCorrectProp', 'Task_Stroop_CongruentCorrectProp',
                               'Task_Stroop_IncongruentCorrectProp', 'Task_Stroop_TotalAvgRT',
                               'Task_Stroop_CongruentAvgRT', 'Task_Stroop_IncongruentAvgRT'
                              ])

In [46]:
## Got all 'SurveyStroopResults' till date in a dataframe
df_stroop_survey = convert_survey_file_to_dataframe(directory, 
                                                'SurveyStroopResults', 
                                                'ActualDueDate')

## Keep Relevant columns only (Wilson, D.)
df = df_stroop_survey
df = df[['ParticipantIdentifier', 'ActualDueDate', 'StartTime', 
         'EndTime', 'ColorSelected', 'Color', 'Text']]

# Create correct, congruent and time columns (Wilson, D.)
df = df.assign(Congruent=lambda x: x.Color == x.Text,
               Correct=lambda x: x.Color == x.ColorSelected,
               Time=lambda x: (x.EndTime - x.StartTime)
              )

df = df.groupby(['ActualDueDate', 
                 'ParticipantIdentifier']).apply(get_stroop_results).reset_index()
df_stroop = df

### PSAT Results
- Calculate TotalTime/Length

In [47]:
## Get all PSAT results till date in a dataframe
df_psat_survey = convert_survey_file_to_dataframe(directory, 
                                                'SurveyPSATResults', 
                                                'ActualDueDate')

## Assign new column with accuracy value (Wilson D.)
df = df_psat_survey
df = df.assign(Task_PSAT_Accuracy=lambda x: x.TotalCorrect/x.Length,
               Task_PSAT_AvgTime=lambda x: x.TotalTime/x.Length
              )

## Keep relevant columns
df_psat = df[['ParticipantIdentifier', 'ActualDueDate', 'Task_PSAT_Accuracy', 'Task_PSAT_AvgTime']]

### Tower of Hanoi Task
- Moves is a string of a list of dictionaries
- Get total time required: Timestamp in last dictionary
- Get number of moves: 1 dict/move so get count of dicts)

**Important data is:**
- PuzzleWasSolved: Indicate completion

In [48]:
## Get all TOH results till date in a dataframe
df_TOH_survey = convert_survey_file_to_dataframe(directory, 
                                                'SurveyTowerOfHanoiResults', 
                                                'ActualDueDate')
df = df_TOH_survey

## Convert string Moves to list of dicts (Wilson D.)
df['MovesList'] = df['Moves'].apply(json.loads)

## Assign new columns (Wilson D.)
df = df.assign(Task_Hanoi_Solved=lambda x: x.PuzzleWasSolved,
               #Task_Hanoi_Time=lambda x: x.MovesList.apply(lambda x: x[-1]['Timestamp']),
               Task_Hanoi_Moves=[len(moves) for moves in df.MovesList] # maybe give this as a multiple on optimality (ideal = 1)?
              )

## Keep relevant columns
df_TOH = df[['ParticipantIdentifier', 'ActualDueDate', 'Task_Hanoi_Solved', #'Task_Hanoi_Time', 
             'Task_Hanoi_Moves']]

### Reaction Time Results
- The important data is:
- ReactionTime
    - Use this to calculate both mean and standard deviation

In [49]:
## Get all Reaction Time results till date in a dataframe
df_rt_survey = convert_survey_file_to_dataframe(directory, 
                                                'SurveyReactionTimeResults', 
                                                'ActualDueDate')

## Seems like RT is in ms ## Convert that to minutes (divide by 1000*60)
df_rt_survey['ReactionTime'] = df_rt_survey.apply(lambda x: (x.ReactionTime)/(60*1000), axis=1)

## Get mean and SD of Reaction Time
df = df_rt_survey
df = pd.DataFrame(df.groupby(['ActualDueDate', 'ParticipantIdentifier'])['ReactionTime'].agg(['mean', 'std'])).reset_index()

## Rename columns (Wilson D.)
df.rename(columns={'mean': 'Task_RT_Mean (mins)', 'std': 'Task_RT_SD'}, inplace=True)

## Get final RT Results dataframe
df_reaction_time = df

### Spatial Span Memory Results
- To capture performance we are using: Score

In [50]:
## Get all Spatial Span Memory results till date in a dataframe
df_ssm_survey = convert_survey_file_to_dataframe(directory, 
                                                'SurveySpatialSpanMemoryResults', 
                                                'ActualDueDate')
## Assign new column with accuracy value (Wilson D.)
df = df_ssm_survey
df = df.assign(Task_SSMemory_Score=lambda x: x.Score)

## Keep relevant columns
df = df[['ParticipantIdentifier', 'ActualDueDate', 'Task_SSMemory_Score']]
df_spatial_span = df

### Get the list of participants

In [51]:
df_participants = pd.read_csv(directory + '/StudyParticipants_20220910.csv')
df_participants["CustomFields"] = df_participants["CustomFields"].apply(json.loads)

# Should have 21 participants
participant_list = []
for index, row in df_participants.iterrows():
    if row["CustomFields"]["exp_version"] == "app_pilot_1":
        participant_list.append(row["ParticipantIdentifier"])

## Get Task Dataframe
For this, we need to get the following df of each good participant:
- df_trail_making
- df_stroop
- df_psat
- df_TOH
- df_reaction_time
- df_spatial_span
    
Merge these 6 dataframes on ParticipantIdentifier and ActualDueDate
- This will give Task Dataframe of one participant

_Do the same for all good participants and concatenate to get final df_

In [52]:
df_task_list = []
good_subjects = participant_list
for item in good_subjects:
    ## Get all relevant DF for the current good subject
    df_trail_making_temp = df_trail_making[df_trail_making.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    df_stroop_temp = df_stroop[df_stroop.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    df_psat_temp = df_psat[df_psat.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    df_TOH_temp = df_TOH[df_TOH.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    df_reaction_time_temp = df_reaction_time[df_reaction_time.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    df_spatial_span_temp = df_spatial_span[df_spatial_span.ParticipantIdentifier.isin([item])].reset_index(drop=True)
    
    ## Merge them to get this subject's task df
    df_task_temp = df_trail_making_temp
    df_task_temp = df_task_temp.merge(df_stroop_temp, how='left', on=['ParticipantIdentifier', 'ActualDueDate'])
    df_task_temp = df_task_temp.merge(df_psat_temp, how='left', on=['ParticipantIdentifier', 'ActualDueDate'])
    df_task_temp = df_task_temp.merge(df_TOH_temp, how='left', on=['ParticipantIdentifier', 'ActualDueDate'])
    df_task_temp = df_task_temp.merge(df_reaction_time_temp, how='left', on=['ParticipantIdentifier', 'ActualDueDate'])
    df_task_temp = df_task_temp.merge(df_spatial_span_temp, how='left', on=['ParticipantIdentifier', 'ActualDueDate'])
    
    df_task_list.append(df_task_temp)

df_task = pd.concat(df_task_list)

In [53]:
df_task

,ParticipantIdentifier,ActualDueDate,Task_Trailmaking_Time,Task_Trailmaking_Errors,Task_Stroop_TotalCorrectProp,Task_Stroop_CongruentCorrectProp,Task_Stroop_IncongruentCorrectProp,Task_Stroop_TotalAvgRT,Task_Stroop_CongruentAvgRT,Task_Stroop_IncongruentAvgRT,Task_PSAT_Accuracy,Task_PSAT_AvgTime,Task_Hanoi_Solved,Task_Hanoi_Moves,Task_RT_Mean (mins),Task_RT_SD,Task_SSMemory_Score
0,48396d3f-058f-4526-b7e7-b260c92774ea,2022-08-11,10.561143,1,0.95,1.0,0.875000,1.158615,1.150792,1.170349,0.000000,3.000000,False,19.0,20.360673,0.000178,460.0
1,48396d3f-058f-4526-b7e7-b260c92774ea,2022-08-11,8.920518,0,0.95,1.0,0.875000,1.158615,1.150792,1.170349,0.000000,3.000000,False,19.0,20.360673,0.000178,460.0
2,48396d3f-058f-4526-b7e7-b260c92774ea,2022-08-12,9.949458,1,0.90,1.0,0.714286,1.058077,1.033104,1.104456,0.433333,2.193591,False,46.0,20.974162,0.000164,400.0
3,48396d3f-058f-4526-b7e7-b260c92774ea,2022-08-12,10.304655,0,0.90,1.0,0.714286,1.058077,1.033104,1.104456,0.433333,2.193591,False,46.0,20.974162,0.000164,400.0
4,48396d3f-058f-4526-b7e7-b260c92774ea,2022-08-12,8.913799,0,0.90,1.0,0.714286,1.058077,1.033104,1.104456,0.433333,2.193591,False,46.0,20.974162,0.000164,400.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,8a5145e5-6977-4beb-b7be-18e4ce6451bb,2022-09-01,9.345251,2,1.00,1.0,1.000000,0.665767,0.651764,0.686772,0.900000,1.247730,False,28.0,4.412037,0.000162,325.0
59,8a5145e5-6977-4beb-b7be-18e4ce6451bb,2022-09-01,10.770921,0,1.00,1.0,1.000000,0.665767,0.651764,0.686772,0.900000,1.247730,False,28.0,4.412037,0.000162,325.0
60,8a5145e5-6977-4beb-b7be-18e4ce6451bb,2022-09-02,9.097224,1,0.95,1.0,0.916667,0.773717,0.625460,0.872555,0.933333,1.293363,True,59.0,5.529071,0.000171,315.0
61,8a5145e5-6977-4beb-b7be-18e4ce6451bb,2022-09-02,6.397452,0,0.95,1.0,0.916667,0.773717,0.625460,0.872555,0.933333,1.293363,True,59.0,5.529071,0.000171,315.0


### Export Task Dataframe

In [15]:
df_task.to_csv('tasks.csv', index=False)